# Encounter Generator

The Encounter Generator provides GMs with a way to quickly generate an encounter that can be incorporated into their campaign.  Simply provide the notebook with a concept and the notebook will do the rest.

> [!IMPORTANT]
> You will need an [.NET 8 SDK](https://dotnet.microsoft.com/en-us/download/dotnet/8.0) and [Polyglot](https://marketplace.visualstudio.com/items?itemName=ms-dotnettools.dotnet-interactive-vscode) to get started with this notebook using .NET Interactive.
> 
>
> To run the LLM prompts and semantic functions, make sure you have an
> - [Azure OpenAI Service Key](https://learn.microsoft.com/azure/cognitive-services/openai/quickstart?pivots=rest-api) or
> - [OpenAI API Key](https://platform.openai.com).

## Step 1 - Define your Concept

Provide your testing context information in `input/user_message.txt`.

In [1]:
// Load all text in ../input/user_message.text
string user_message = await System.IO.File.ReadAllTextAsync("input/user_message.txt");

// Load all text in ../input/system_message.text
string system_message = await System.IO.File.ReadAllTextAsync("input/system_message.txt");


## Step 2 - Initial Configuration

Run the following cells to configure your AI generative settings.


Choose whether you wish to use the OpenAI or Azure OpenAI service.

In [2]:
bool useAzureOpenAI = false;

Execute the following code which will ask a few questions and save the settings to a local
`settings.json` configuration file, under the [config](config) folder. You can
also edit the file manually if you prefer. **Please keep the file safe.**

In [3]:
#!import config/Settings.cs

await Settings.AskAzureEndpoint(useAzureOpenAI);
await Settings.AskModel(useAzureOpenAI);
await Settings.AskApiKey(useAzureOpenAI);

// Uncomment this if you're using OpenAI and need to set the Org Id
await Settings.AskOrg(useAzureOpenAI);

Settings: OK: AI model configured [config/settings.json]
Settings: OK: API key configured [config/settings.json]


## Step 3 - Instantiate Services

In [4]:
// Remove previously installed nuget packages
#r "nuget: Microsoft.SemanticKernel"
#r "nuget: System.Numerics.Tensors"
#r "nuget: SkiaSharp"

#!import config/Settings.cs
#!import config/Utils.cs
#!import config/SkiaUtils.cs

using System.ComponentModel;
using System.Numerics.Tensors;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.Embeddings;
using Microsoft.SemanticKernel.TextToImage;

#pragma warning disable SKEXP0001, SKEXP0010

var builder = Microsoft.SemanticKernel.Kernel.CreateBuilder();

// Configure AI backend used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

var httpClient = new HttpClient
{
    Timeout = TimeSpan.FromMinutes(10),
};


if (useAzureOpenAI)
{
    builder.AddAzureOpenAITextEmbeddingGeneration("text-embedding-ada-002", azureEndpoint, apiKey);
    builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey, httpClient: httpClient);
    builder.AddAzureOpenAITextToImage("dall-e-3", azureEndpoint, apiKey);
}
    
else
{
    builder.AddOpenAITextEmbeddingGeneration("text-embedding-ada-002", apiKey, orgId);
    builder.AddOpenAIChatCompletion(model, apiKey, orgId, httpClient: httpClient);
    builder.AddOpenAITextToImage(apiKey, orgId);
}

var pluginsDirectory = Path.Combine(System.IO.Directory.GetCurrentDirectory(), "plugins");

var kernel = builder.Build();

// Get plugin functions
var pluginFunctions = kernel.ImportPluginFromPromptDirectory(Path.Combine(pluginsDirectory, "LocationPlugin"));

// Create the output directory
var outputDirectory = Path.Combine(System.IO.Directory.GetCurrentDirectory(), "output");
if (!Directory.Exists(outputDirectory))
{
    Directory.CreateDirectory(outputDirectory);
}

// Get chat completion service instance
var chatCompletionService = kernel.GetRequiredService<IChatCompletionService>();

// Get AI service instance used to generate images
var dallE = kernel.GetRequiredService<ITextToImageService>();

// Get AI service instance used to extract embedding from a text
var textEmbedding = kernel.GetRequiredService<ITextEmbeddingGenerationService>();

OpenAIPromptExecutionSettings openAIPromptExecutionSettings = new() 
{
    ToolCallBehavior = ToolCallBehavior.AutoInvokeKernelFunctions
};

var history = new ChatHistory();


Installed Packages Microsoft.SemanticKernel, 1.16.2 SkiaSharp, 2.88.8 System.Numerics.Tensors, 8.0.0

Loading extensions from `C:\Users\n01572960\.nuget\packages\skiasharp\2.88.8\interactive-extensions\dotnet\SkiaSharp.DotNet.Interactive.dll`

## Step 4 - Generate the Content

All generated output will be found in the `output` folder.  If no files exist in this folder post execution, expand this section and review the code execution results for failure.

Generate the encounter.

In [5]:
#pragma warning disable SKEXP0001

// Add the system message to the chat history
history.AddSystemMessage(system_message);

// Add the user message to the chat history
history.AddUserMessage(user_message);

// 1. Get the response from the AI with automatic function calling
var result = await chatCompletionService.GetChatMessageContentAsync(
    history,
    executionSettings: openAIPromptExecutionSettings,
    kernel: kernel);

// Add the message from the agent to the chat history
history.AddMessage(result.Role, result.Content ?? string.Empty);

The following code outputs the history to the console for debugging purposes.

In [6]:
#pragma warning disable SKEXP0001

// Examine the chat history and output a summary to the console
foreach (var message in history.ToArray<ChatMessageContent>())
{
    // Limit the output to 100 characters
    var formattedMessage = (message.Content?.Length > 100 ? message.Content.Substring(0, 100) + "..." : message.Content ?? string.Empty).Replace("\n", " ").Replace("\r", " ").Replace("\t", " ");

    if (message.Role == AuthorRole.Tool)
    {
        var functionResult = (FunctionResultContent) message.Items.Where<KernelContent>(x => x is Microsoft.SemanticKernel.FunctionResultContent).FirstOrDefault();
        Console.WriteLine($"{functionResult?.PluginName ?? string.Empty}/{functionResult?.FunctionName ?? string.Empty}:");
        Console.WriteLine($"\t{formattedMessage}");
    }
    else if (message.Role == AuthorRole.System)
    {
        Console.WriteLine("System: ");
        Console.WriteLine($"\t{formattedMessage}");
    }
    else if (message.Role == AuthorRole.User)
    {
        Console.WriteLine("User: ");
        Console.WriteLine($"\t{formattedMessage}");
    }
    else if (message.Role == AuthorRole.Assistant)
    {
        Console.WriteLine("Assistant: ");
        foreach (var item in message.Items.ToArray<KernelContent>())
        {
            if (item is FunctionCallContent functionResult)
            {
                functionResult = (FunctionCallContent) item;
                Console.WriteLine($"\tCalling {functionResult.PluginName}/{functionResult.FunctionName}");
            }
            else if (!string.IsNullOrEmpty(formattedMessage))
            {
                Console.WriteLine($"\t{formattedMessage}");
            }
        }
    }
    else
    {
        Console.WriteLine($"Unknown: {formattedMessage}");
    }
}

System: 
	You are a game master for a Dungeons and Dragons campaign that specializes in creating fantastic eco...
User: 
	Please generate a location based on the following information: * The town of Hammersend is a port to...
Assistant: 
	Calling LocationPlugin/Location
LocationPlugin/Location:
	## Location: Hammersend  ### 1. Define the Purpose of the Location - **Role in the Story**: Hammerse...
Assistant: 
	Calling LocationPlugin/NPC
	Calling LocationPlugin/Encounter
	Calling LocationPlugin/LootTable
LocationPlugin/NPC:
	### NPC 1: Seraphina the Sea Witch   **Appearance**: Seraphina is a tall, ethereal woman with long, ...
LocationPlugin/Encounter:
	## Encounter: The Cult of the Sea Witch  ### Purpose and Context - **Narrative Role**: This encounte...
LocationPlugin/LootTable:
	| D20 Roll | Item Name               | Low Value Description                                   | Med...
Assistant: 
	# Hammersend: The Port Town of Secrets  ## Location Overview  ### Define the Purpose of the

The generated content will be more useful to us if we can structure it into a data format that can be consumed by other systems.

In [4]:
// Ensure the output directory exists
Directory.CreateDirectory("output");

// Delete all files and directories in the output directory
foreach (var file in Directory.GetFiles("output"))
{
    File.Delete(file);
}

foreach (var dir in Directory.GetDirectories("output"))
{
    Directory.Delete(dir, true);
}

// Save the result to the output directory
await File.WriteAllTextAsync(Path.Combine(outputDirectory, "encounter.md"), result.Content);